# 👷 Serving a Model into Production

### Import python libraries

In [1]:
import mlflow
import numpy as np
from fastapi import FastAPI
from pydantic import BaseModel

## Download model from MLFlow

In [2]:
MLFLOW_URI = "http://mlflow:5000"
MODEL_URI = "models:/titanic-linear-regression/Staging"


# Load the MLflow model into memory
mlflow.set_tracking_uri(uri=MLFLOW_URI)
model = mlflow.pyfunc.load_model(model_uri=MODEL_URI)

## Create the API

### Define the API

In [3]:
# Define the FastAPI app
app = FastAPI()

# Define the input data schema using Pydantic BaseModel
class InputData(BaseModel):
    sex: int
    age: float
    fare: float
    embarked: int


# Define the API endpoint
@app.post('/predict')
def predict(input_data: InputData):
    # Process the input data
    features = np.array([
        input_data.sex,
        input_data.age,
        input_data.fare,
        input_data.embarked
    ]).reshape(1, -1)

    # Use the MLflow model to perform inference
    prediction = model.predict(features)

    # Return the inference results
    return {'prediction': prediction.tolist()}

### Launch the API

In [ ]:
import uvicorn

if __name__ == "__main__":
    config = uvicorn.Config(app)
    server = uvicorn.Server(config)
    await server.serve()